**This Jupyter Notebook covers data acquisition from College Confidential**

## 1.1 Importing packages needed to scrape and save serialized data

In [ ]:
#<font color='blue'>*robots.txt*</font>

In [26]:
from bs4 import BeautifulSoup
from datetime import datetime 
import json
import os
import requests


## # 1.2 Checking website policy

In [27]:
robots_txt = open_url('https://www.collegeconfidential.com/robots.txt')

User-agent: *

Disallow: /cgi-bin

Disallow: /wp-admin

Disallow: /wp-includes

Disallow: /wp-content/plugins

Disallow: /wp-content/cache

Disallow: /wp-content/themes

Disallow: /vibe

Disallow: /trackback

Disallow: /feed

Disallow: /comments

Disallow: /category/*/*

Disallow: */trackback

Disallow: */feed

Disallow: */comments

Disallow: /*?*

Disallow: /*?

Allow: /wp-content/uploads



# Google Image

User-agent: Googlebot-Image

Disallow:

Allow: /*



# Google AdSense

User-agent: Mediapartners-Google*

Disallow:

Allow: /*



# Internet Archiver Wayback Machine

User-agent: ia_archiver

Disallow: /



# digg mirror

User-agent: duggmirror

Disallow: /


In [17]:
requests.get('https://talk.collegeconfidential.com/university-pennsylvania/').status_code

200

In [4]:
penn_url = 'https://talk.collegeconfidential.com/university-pennsylvania//p1'

In [5]:
response = requests.get(penn_url)

In [6]:
response.status_code

200

In [11]:
page = response.text
print(page[0:250])

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8">
<meta http-equiv="X-UA-Compatible" content="IE=edge">
<style>
    .DataList.CategoryList .Options {
        display: none;
    }
</style><style>

ul.AcceptedAnswers .Item {
   background:


In [6]:
soup = BeautifulSoup(page, 'html5')

In [7]:
# by inspection, there are 250 pages
page_list = []
for i in range(1, 251):
    page_list.append('https://talk.collegeconfidential.com/university-pennsylvania//p{}'.format(i))

In [8]:
def get_one_page_titles(soup):
    """
    This function scrapes the thread titles displayed on one page of results
    
    """
    topics_json = []
    for div in soup.find_all('div',  class_='Title'):
        for link in div.find_all('a'):
            topics_json.append({'topic': link.text, 
                                'url': link.get("href")})
    return topics_json

In [9]:
def get_all_topics(page_list):
    count = 1
    topics_json = []
    
    for url in page_list:    
        response = requests.get(url)
        page = response.text
        soup = BeautifulSoup(page, 'html5')
        topics_json.append(get_one_page_titles(soup))

        if count % 25 == 0:
            print("Finished page {}".format(count))
        count += 1
    topics_json = [item for sublist in topics_json for item in sublist]
    return topics_json

In [10]:
#topics_json = get_all_topics(page_list)

#with open("penn_topics.json", 'w', encoding='utf-8') as outfile:
    #json.dump(topics_json, outfile)

In [11]:
with open("penn_topics.json", 'r') as f:
    topics_json = json.load(f)
    f.close()

In [12]:
def page_comments(url):
    comments = []
    
    page = requests.get(url, timeout=100).text
    soup = BeautifulSoup(page, 'html5')
    
    dates = []
    comments = []
    user_id = []
    user_names = []
    user_thread_count = []
    user_comment_count = []
    lst = []
        
    # comment dates broken by time component
    for div in soup.find_all('div', class_ ="Meta Discussion DiscussionInfo"):
        for i in div.find_all('span', class_="MItem DateCreated"):
            for j in i.find_all('time'):
                split = j.get('title').split()
                dates.append({
                            'month': split[0],
                            'day': split[1],
                            'year': split[2],
                            'time': split[3]
                            })
            
    # comment text
    for div in soup.find_all('div', class_ = "Message userContent"):
        comments.append(div.text)    
    # user name
    for div in soup.find_all('div', class_ ="AuthorWrap"):
        for span in div.find_all('span', class_="Author"):
            user_names.append(span.text.replace('\n', ''))
    # user id
    for div in soup.find_all('div', class_ ="AuthorWrap"):
        for i in div.find_all('a'):
            user_id.append(i.get("data-userid"))
    # user thread count
    for div in soup.find_all('div', class_ ="AuthorWrap"):
        for span in div.find_all('span', class_ = "MItem CountDiscussions"):
            user_thread_count.append(span.text.split()[0])      
    # user comment count
    for div in soup.find_all('div', class_ ="AuthorWrap"):
        for span in div.find_all('span', class_ = "MItem CountComments"):
            user_comment_count.append(span.text.split()[0])
            
    # make json
    for i in range(0, len(comments)):
        lst.append({
            'date' :dates[i],
            'comment': comments[i],
            'user_id': user_id[i],
            'user_name': user_names[i],
            'user_thread_count': user_thread_count[i],
            'user_comment_count': user_comment_count[i],
                    })
    return lst  

In [13]:
# return flatten list of length number of comments
def topic_comments(url):    
    all_comments = []
    return_list = []
    
    i = 1
    
    base_url = url.split(".html")[0]
    test_url = base_url + "-p{}".format(i) + ".html"
    response = requests.get(test_url, timeout=100)
    
    while response.status_code != 404:
        test_url = base_url + "-p{}".format(i) + ".html"
        response = requests.get(test_url, timeout=100)
        all_comments.append(page_comments(test_url))
        i += 1
   
    flat_list = [item for sublist in all_comments for item in sublist]
    return flat_list
    

In [14]:
def get_forum_comms(topics_json, lower, upper):
    forum = []
    
    i = lower
    for topic in topics_json:
        start_time = datetime.now()
        thread = topic['topic']
        url = topic['url']
        
        try:
            forum.append({
                        'topic': thread,
                        'url': url,
                        'comments' : topic_comments(url)
                         })
        except:
            print("Thread {} failed".format(i))
            
        
        time_elapsed = datetime.now() - start_time
        print("{}: Thread {}:".format(datetime.now(), i), 'Time elapsed {}(hh:mm:ss.ms)'.format(time_elapsed))

        i += 1
    
    # for windows file structure 
    filename = os.getcwd() + "\\penn_data\\{}_to_{}.json".format(lower, upper)
     
    with open(filename, 'w', encoding = 'utf-8') as outfile:
        json.dump(forum, outfile)
        
    print("saved {}".format(filename))

In [15]:
low = 0
high = 100

lowers = []
uppers = []
while low < len(topics_json):
    lowers.append(low)
    low += 100

while high < len(topics_json):
    uppers.append(high)
    high +=  100

In [16]:
uppers.append(len(topics_json))

In [17]:
lims = list(zip(lowers, uppers))

In [18]:
for i, j in lims:
    print("get_forum_comms(topics_json[{}:{}], {}, {})".format(i, j, i, j - 1))

get_forum_comms(topics_json[0:100], 0, 99)
get_forum_comms(topics_json[100:200], 100, 199)
get_forum_comms(topics_json[200:300], 200, 299)
get_forum_comms(topics_json[300:400], 300, 399)
get_forum_comms(topics_json[400:500], 400, 499)
get_forum_comms(topics_json[500:600], 500, 599)
get_forum_comms(topics_json[600:700], 600, 699)
get_forum_comms(topics_json[700:800], 700, 799)
get_forum_comms(topics_json[800:900], 800, 899)
get_forum_comms(topics_json[900:1000], 900, 999)
get_forum_comms(topics_json[1000:1100], 1000, 1099)
get_forum_comms(topics_json[1100:1200], 1100, 1199)
get_forum_comms(topics_json[1200:1300], 1200, 1299)
get_forum_comms(topics_json[1300:1400], 1300, 1399)
get_forum_comms(topics_json[1400:1500], 1400, 1499)
get_forum_comms(topics_json[1500:1600], 1500, 1599)
get_forum_comms(topics_json[1600:1700], 1600, 1699)
get_forum_comms(topics_json[1700:1800], 1700, 1799)
get_forum_comms(topics_json[1800:1900], 1800, 1899)
get_forum_comms(topics_json[1900:2000], 1900, 1999)
get_

In [19]:
#get_forum_comms(topics_json[0:99], 0, 99)

In [20]:
#get_forum_comms(topics_json[100:199], 100, 199)

In [21]:
#get_forum_comms(topics_json[200:299], 200, 299)

In [22]:
#get_forum_comms(topics_json[300:399], 300, 399)

In [23]:
len(topics_json[0:100])

100

In [24]:
#get_forum_comms(topics_json[400:500], 400, 500)

In [25]:
#get_forum_comms(topics_json[500:600], 500, 600)

In [26]:
#get_forum_comms(topics_json[600:700], 600, 700)

In [27]:
#get_forum_comms(topics_json[700:800], 700, 800)

In [28]:
#get_forum_comms(topics_json[800:900], 800, 900)

In [29]:
#get_forum_comms(topics_json[900:1000], 900, 1000)

In [30]:
#get_forum_comms(topics_json[1000:1100], 1000, 1100)

In [31]:
#get_forum_comms(topics_json[1100:1200], 1100, 1200)

In [32]:
#get_forum_comms(topics_json[1200:1300], 1200, 1300)

In [33]:
#get_forum_comms(topics_json[1300:1400], 1300, 1400)

In [34]:
#get_forum_comms(topics_json[1400:1500], 1400, 1500)

In [35]:
#get_forum_comms(topics_json[1500:1600], 1500, 1600)

In [36]:
#get_forum_comms(topics_json[1600:1700], 1600, 1700)

In [37]:
#get_forum_comms(topics_json[1700:1800], 1700, 1800)

In [38]:
#get_forum_comms(topics_json[1800:1900], 1800, 1900)

In [39]:
#get_forum_comms(topics_json[1900:2000], 1900, 2000)

In [40]:
#get_forum_comms(topics_json[2000:2100], 2000, 2100)

In [41]:
#get_forum_comms(topics_json[2100:2200], 2100, 2200)

In [42]:
#get_forum_comms(topics_json[2200:2300], 2200, 2300)

In [43]:
#get_forum_comms(topics_json[2300:2400], 2300, 2400)

In [44]:
#get_forum_comms(topics_json[2400:2500], 2400, 2500)

In [45]:
#get_forum_comms(topics_json[2500:2600], 2500, 2600)

In [46]:
#get_forum_comms(topics_json[2600:2700], 2600, 2700)

In [47]:
#get_forum_comms(topics_json[2700:2800], 2700, 2800)

In [48]:
#get_forum_comms(topics_json[2800:2900], 2800, 2900)

In [49]:
#get_forum_comms(topics_json[2900:3000], 2900, 3000)

In [50]:
#get_forum_comms(topics_json[3000:3100], 3000, 3100)

In [51]:
#get_forum_comms(topics_json[3100:3200], 3100, 3200)

In [52]:
#get_forum_comms(topics_json[3200:3300], 3200, 3300)

In [53]:
#get_forum_comms(topics_json[3300:3400], 3300, 3400)

In [54]:
#get_forum_comms(topics_json[3400:3500], 3400, 3500)

In [55]:
#get_forum_comms(topics_json[3500:3600], 3500, 3600)

In [56]:
#get_forum_comms(topics_json[3600:3700], 3600, 3700)

In [57]:
#get_forum_comms(topics_json[3700:3800], 3700, 3800)

In [58]:
#get_forum_comms(topics_json[3800:3900], 3800, 3900)

In [59]:
#get_forum_comms(topics_json[3900:4000], 3900, 4000)

In [60]:
#get_forum_comms(topics_json[4000:4100], 4000, 4100)

In [61]:
#get_forum_comms(topics_json[4100:4200], 4100, 4200)

In [62]:
#get_forum_comms(topics_json[4200:4300], 4200, 4300)

In [63]:
#get_forum_comms(topics_json[4300:4400], 4300, 4400)

In [64]:
#get_forum_comms(topics_json[4400:4500], 4400, 4500)

In [65]:
#get_forum_comms(topics_json[4500:4600], 4500, 4600)

In [66]:
#get_forum_comms(topics_json[4600:4700], 4600, 4700)

In [67]:
#get_forum_comms(topics_json[4700:4800], 4700, 4800)

In [68]:
#get_forum_comms(topics_json[4800:4900], 4800, 4900)

In [69]:
#get_forum_comms(topics_json[4900:4993], 4900, 4993)